In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import _pickle as pkl
from numpy import *
from sklearn import metrics


### 挑选了LDL里面的aa_knn，较为简单，不知道会不会影响结果

In [2]:
def aa_knn(trainFeature, trainLabel, testFeature, testLabel, k):
    rows = testFeature.shape[0]
    cols = testFeature.shape[1]

    train_num = trainFeature.shape[0]
    dist = np.zeros(shape=(train_num))
    prediction = np.zeros(shape=(rows,trainLabel.shape[1]))

    for i in range(rows):
        test = testFeature[i,:]
        for j in range(train_num):
            train = trainFeature[j,:]
            V = test - train
            sqV = V**2
            sqDis = sqV.sum(axis=0)
            dist[j] = math.sqrt(sqDis)

        indice = dist.argsort()
        sort_dis = sorted(dist)
        sum_label = np.zeros(shape=trainLabel.shape[1])
        for l in range(k):
            sum_label = sum_label + trainLabel[l,:]

        prediction[i,:] = sum_label/k
    
    idx = prediction.argmax(axis=1)
    prediction = (idx[:,None] == np.arange(prediction.shape[1])).astype(int)

    score = metrics.accuracy_score(prediction, testLabel)

    return prediction, score 

In [ ]:
def tmc_error(mem):
    

In [69]:
def tmc_shap(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, iterations, sources=None):
    if sources is None:
        sources = {i: np.array([i]) for i in range(len(trainFeature))}
    elif not isinstance(sources, dict):
        sources = {i: np.where(sources == i)[0] for i in set(sources)}

    marginals = []
    mem_tmc = np.zeros((0,len(trainFeature)))

    for i in range(iterations):

        idxs = np.random.permutation(len(sources))
        marginal_contribs = np.zeros(len(trainFeature))

        #x_batch = np.zeros((0,) + tuple(trainFeature.shape[1:]))
        #y_batch = np.zeros(0, int)
        x_batch = trainFeature[idxs[0]]
        y_batch = trainLabel[idxs[0]]

        truncation_counter = 0

        #源代码里面是随机抽取测试数据，进行100次实验获得到的平均socre，这里直接设置为零
        new_score = 0
  
        for sh, idx in enumerate(idxs):
            old_score = new_score

            if sh == 0 :
                a=0
            else:
                x_batch = np.vstack((x_batch, trainFeature[idx]))
                y_batch = np.vstack((y_batch, trainLabel[idx]))

            if x_batch.shape[0] >= 2:
                _, new_score = aa_knn(trainFeature, trainLabel, testFeature, testLabel, k)

            marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
            mean_score = 0.18

            distance_to_full_score = np.abs(new_score - mean_score)
            if distance_to_full_score <= tolerance * mean_score:
                truncation_counter += 1
                if truncation_counter > 5:
                    break
            else:
                truncation_counter = 0
        
        mem_tmc = np.concatenate([mem_tmc, np.reshape(marginal_contribs, (1,-1))])

    value_tmc = np.mean(mem_tmc,0)

    return value_tmc,mem_tmc


### 读取数据

In [4]:
file_feature = open("ldlFeature.txt",'r')
feature = []

for line in file_feature:
    one_fea = list(map(lambda x : float(x), line.split()))
    feature.append(one_fea)


file_label = open("binaryLabel.txt",'r')
label = []

for line in file_label:
    one_label = list(map(lambda x : float(x), line.split()))
    label.append(one_label)

### 设置训练数据和测试数据

In [144]:
trainFeature = array(feature[:100])
trainLabel = array(label[:100])

testFeature = array(feature[170:])
testLabel = array(label[170:])

In [70]:
shap,tmc = tmc_shap(0.1, trainFeature, trainLabel, testFeature, testLabel, 5, 20, sources=None)

In [145]:
for i in range(1):
    new_value = np.zeros((0,len(trainFeature)))

    #new_value,_ = tmc_shap(0.1, trainFeature, trainLabel, testFeature, testLabel, 5, 1, sources=None)
    for k in range(10):
        value_one,_ = tmc_shap(0.1, trainFeature, trainLabel, testFeature, testLabel, 5, 100, sources=None)
        new_value = np.concatenate([new_value,np.reshape(value_one, (1,-1))])
        
    new_value = np.mean(new_value,0)
    print(new_value)
    
    mod = True

    trainLabel_tolist = trainLabel.tolist()
    idx_high = int(np.where(trainLabel[i]==np.max(trainLabel[i]))[0])
    idx_low = 1 - idx_high

    mod_count = 0

    while mod:
        #a = np.abs(trainLabel[i,0] - trainLabel[i,1])
        old_value = new_value

        #new_value,_ = tmc_shap(0.3, trainFeature, trainLabel, testFeature, testLabel, 5, 10, sources=None)

        trainLabel_tolist[i][idx_high] -= 0.01
        trainLabel_tolist[i][idx_low] = 1 - trainLabel_tolist[i][idx_high]

        if trainLabel_tolist[i][idx_high] <= 0:
            mod = False

        trainLabel = array(trainLabel_tolist)
        
        for k in range(10):
            new_value,_ = tmc_shap(0.1, trainFeature, trainLabel, testFeature, testLabel, 5, 100, sources=None)
            
            print(new_value[i],old_value[i])
            if new_value[i] >= old_value[i]:
                mod_count += 1

        print(mod_count)
        if mod_count > 1:
            mod_count = 0
        else:
            mod = False
         


[0.00297297 0.00237838 0.00356757 0.00267568 0.00237838 0.00148649
 0.00356757 0.00356757 0.00386486 0.00237838 0.00237838 0.00267568
 0.00267568 0.00208108 0.00445946 0.00267568 0.00267568 0.00505405
 0.00327027 0.00356757 0.00386486 0.00237838 0.00267568 0.00356757
 0.00327027 0.00356757 0.00237838 0.00356757 0.00327027 0.00386486
 0.00178378 0.00683784 0.00327027 0.00356757 0.00208108 0.00267568
 0.00208108 0.00118919 0.00356757 0.00178378 0.00445946 0.00356757
 0.00327027 0.00267568 0.00118919 0.00237838 0.00267568 0.00327027
 0.00475676 0.00267568 0.00237838 0.00237838 0.00297297 0.00297297
 0.00297297 0.00475676 0.00297297 0.00327027 0.00178378 0.00267568
 0.00297297 0.00356757 0.00148649 0.00356757 0.00148649 0.00297297
 0.00356757 0.00089189 0.00297297 0.00327027 0.00208108 0.00386486
 0.00386486 0.00564865 0.00297297 0.00267568 0.00416216 0.00237838
 0.00178378 0.00148649 0.00237838 0.00237838 0.00297297 0.00327027
 0.00267568 0.00297297 0.00327027 0.00386486 0.00237838 0.0026

KeyboardInterrupt: 

In [146]:
print(trainLabel[0])

[0.7 0.3]


In [76]:
def one_iteration(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None):
    if sources is None:
        sources = {i: np.array([i]) for i in range(len(trainFeature))}
    elif not isinstance(sources, dict):
        sources = {i: np.where(sources == i)[0] for i in set(sources)}

    idxs = np.random.permutation(len(sources))
    marginal_contribs = np.zeros(len(trainFeature))

    #x_batch = np.zeros((0,trainFeature.shape[1]))
    #y_batch = np.zeros(0,float)

    x_batch = trainFeature[idxs[0]]
    y_batch = trainLabel[idxs[0]]

    truncation_counter = 0

    #源代码里面是随机抽取测试数据，进行100次实验获得到的平均socre，这里直接设置为零
    new_score = 0

    for sh, idx in enumerate(idx):
        old_score = new_score

        #x_batch = np.concatenate(x_batch, trainLabel[sources[idx]])
        #y_batch = np.concatenate(y_batch, trainFeature[sources[idx]])
        if sh == 0 :
                a=0
        else:
            x_batch = np.vstack((x_batch, trainFeature[idx]))
            y_batch = np.vstack((y_batch, trainLabel[idx]))
        if len(set(y_batch)) >= 2:
            _,new_score = aa_knn(trainFeature, trainLabel, testFeature, testLabel, k)

        marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
        mean_score = 0.18

        distance_to_full_score = np.abs(new_score - mean_score)
        if distance_to_full_score <= tolerance * mean_score:
            truncation_counter += 1
            if truncation_counter > 5:
                break
        else:
            truncation_counter = 0
    return marginal_contribs
'''
        if len(set(y_batch)) >= 2:
            new_score = aa_knn()

        marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
        mean_score = 0.18

        distance_to_full_score = np.abs(new_score - mean_score)
        if distance_to_full_score <= tolerance * mean_score:
            truncation_counter += 1
            if truncation_counter > 5:
                break
        else:
            truncation_counter = 0
    return marginal_contribs
'''

In [ ]:
 """
        for sh, idx in enumerate(idxs):
            old_score = new_score

            if sh == 0 :
                a=0
            else:


                x_batch = np.vstack((x_batch, trainFeature[idx]))
                y_batch = np.vstack((y_batch, trainLabel[idx]))
"""
